In [1]:
import mesa #for MESA
import seaborn as sns #data visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Agent and other elements on the grid

In [13]:
from mesa import Model
import random

def check_if_within_AccessPoint_radius(AccessPoint_pos, agent_pos, radius):
  
  distance = ((agent_pos[0] - AccessPoint_pos[0])**2 + (agent_pos[1] - AccessPoint_pos[1])**2)**0.5
  return distance <= radius



class AccessPoint():
  """
  Class that represents a AccessPoint
  
  Attributes:
    pos_x (int): the x coord of the AccessPoint
    pos_y (int): the y coord of the AccessPoint
    radius (int): the radius of the AccessPoint
    capacity (int): the capacity of the AccessPoint, i.e., how many agents need to be in the radius to solve the AccessPoint
  """
  def __init__(self, unique_id:int,  pos_x:int, pos_y:int, radius: int, capacity: int) -> None:
    """
    Initialize a Tast
  
    Args:
      pos_x (int): the x coord of the AccessPoint
      pos_y (int): the y coord of the AccessPoint
      radius (int): the radius of the AccessPoint
      capacity (int): the capacity of the AccessPoint, i.e., how many agents need to be in the radius to solve the AccessPoint
    """
    # super().__init__(unique_id, model)
    self.unique_id = unique_id
    self.pos_x = pos_x
    self.pos_y = pos_y
    self.capacity = capacity  #how many agents should be within the radius
    self.radius = radius
    self.radius_points = self.get_circle_points()
    
    #for coloring the radius
    self._shades_of_gray=["#D3D3D3", "#A9A9A9", "#808080", "#696969", "#000000" ]
    self.radius_color = self._shades_of_gray[random.randint(0,len(self._shades_of_gray)-1)]
  
  def get_circle_points(self):
    points = []
    for x in range(self.pos_x - self.radius, self.pos_x + self.radius + 1):
      for y in range(self.pos_y - self.radius, self.pos_y + self.radius + 1):
        # Check if the distance from (x, y) to the center is less than or equal to the radius
        if ((x - self.pos_x) ** 2 + (y - self.pos_y) ** 2) ** 0.5 <= self.radius:
          if((x,y) != (self.pos_x, self.pos_y)):
            points.append((x, y))
    return points

class Radius:
  """
  This is only to represent the radius of each AccessPoint for visualization purposes. It does not extend the main
  mesa.Agent AccessPoint as it is not agent, although, it belongs to a AccessPoint
  """
  def __init__(self, accessPoint: AccessPoint) -> None:
    self.accessPoint = accessPoint #which AccessPoint the radius point belongs to
    

class SwarmIntelligenceAgent(mesa.Agent):
  def __init__(self, unique_id: int, model: Model, speed: int) -> None:
    super().__init__(unique_id, model)
    self.random.seed()
    self.speed = speed
    self.connected = False
    #let's keep track of how many times an agent was connected during a simulations
    self.num_times_connected = 0
    

  def step(self):
    self.move()    

  def move(self):
    if self.connected:
      #agent is in AccessPoint radius cannot move until the AccessPoint is solved and disappeared
      # print(f"Agent #{self.unique_id} is using an AccessPoint now...")
      cell_contents = self.model.grid.get_cell_list_contents(self.pos)
      for item in cell_contents:
        if isinstance(item, Radius) or isinstance(item, AccessPoint):
          return
        else:
          self.connected = False

    #get the possible steps, i.e., the surrounding cells
    possible_steps = self.model.grid.get_neighborhood(
      self.pos,
      moore=True, #includes all 8 surrounding squares (other is von Neumann, only left/right,up/down)
      include_center=False #staying in at the same space is not allowed
    )
    #decide the next position
    new_position = self.random.choice(possible_steps)
    #place the agent to the new position
    self.model.grid.move_agent(self, new_position)

    # check if the cell containes a AccessPoint or radius (prolly cannot get AccessPoint without radius first)
    cell_contents = self.model.grid.get_cell_list_contents(new_position)
    for item in cell_contents:
      accessPoint = None
      if isinstance(item, Radius):
        accessPoint = item.accessPoint
      elif  isinstance(item,AccessPoint):
        accessPoint = item

      if accessPoint is not None:
        # print(f"Agent #{self.unique_id} is within the radius of AccessPoint #{accessPoint.unique_id}")
        self.connected = True
        self.num_times_connected += 1
        #decrease capacity of AccessPoint to indicate one agent is there to solve
        accessPoint.capacity -= 1
        #if accesspoint is dead, then remove it
        if accessPoint.capacity == 0:
          self.model.remove_accessPoint(accessPoint)
          #free up agents
          self.connected = False
          #create a new accesspoint somewhere else
          # print(f"Respawn an access point")
          self.model.add_accessPoint()
      

# Model

In [21]:
def compute_connected_ratio(model):
  num_agents = model.num_agents
  connected_agents = 0
  for agent in model.schedule.agents:
    if(agent.connected):
      connected_agents+=1
    # else:
    #   if(connected_agents>1):
    #     connected_agents-=1
    #   else:
    #     connected_agents=0
  # print(connected_agents)
  return (connected_agents/num_agents)*100


class SwarmIntelligenceModel(mesa.Model):
  def __init__(self, 
               num_agents: int, 
               grid_x: int, 
               grid_y: int,
               accessPoint_radius_max: int,
               num_accessPoints: int,
               accessPoint_random_radius=False) -> None:
    super().__init__()
    self.num_agents = num_agents
    self.num_accessPoints = num_accessPoints
    self.accessPoint_random_radius=accessPoint_random_radius
    
    
    #let's maintain a dictionary of accessPoints with their data to keep track of them easier
    self.accessPoints = dict()
    #print(f"Initalizing model with {self.num_agents} number of agents...")
    
    #setting AccessPoint radius max
    self.accessPoint_radius_max = accessPoint_radius_max
    # Create scheduler and assign it to the model
    self.schedule = mesa.time.RandomActivation(self)
   
    #create a grid where agents can move
    self.grid = mesa.space.MultiGrid(width=grid_x, height=grid_y, torus=True) #sets grid to be a toroidal, so no edge is there:left continues on the right, and vice versa in all directions


    """Additional variable for batched runs"""
    self.running = True

    """Add data collector(s)"""
    self.datacollector = mesa.DataCollector(
      model_reporters={"Connection Tracker":compute_connected_ratio},
      agent_reporters={"Connected status":"connected"}
    )
    
    #create agents
    for i in range(self.num_agents):
      a = SwarmIntelligenceAgent(unique_id=0, model=self, speed=1)
      #add agent to scheduler
      self.schedule.add(a)
      #add agent to a random cell on the grid
      x = self.random.randrange(self.grid.width)
      y = self.random.randrange(self.grid.height)
      # print(f"Adding an agent to {(x,y)}")
      self.grid.place_agent(a, (x,y))
    
    for i in range(self.num_accessPoints):
      success = self.add_accessPoint()
      if not success:
        success = self.add_accessPoint()

  def add_accessPoint(self): 
    #create a AccessPoint
    self.random.seed()
    t_id = self.random.randrange(100)
    if(self.accessPoint_random_radius):
      t_radius = self.random.randrange(1,self.accessPoint_radius_max)
    else:
      t_radius = self.accessPoint_radius_max
    t_capacity = t_radius+1
    #we need to subtract the radius from the width and height to avoid IndexOutOfBounds error
    #as we do not implement the toroidal feature for the AccessPoints
    t_x = self.random.randrange(t_radius, self.grid.width - t_radius)
    t_y = self.random.randrange(t_radius, self.grid.height - t_radius)

    #Check if there is an accesspoint already at the random spot 
    cell_content = self.grid.get_cell_list_contents((t_y, t_x))
    for c in cell_content:
      if isinstance(c, Radius) or isinstance(c,AccessPoint):
        # print(f"There is already an accesspoint nearby {(t_y,t_x)}...add it to somewhere else")
        return False

    #add access point
    task = AccessPoint(unique_id=t_id, pos_x=t_x, pos_y=t_y, radius=t_radius, capacity=t_capacity)
    self.grid[t_y][t_x].append(task)
    self.accessPoints[task]={
      "pos_x": t_x,
      "pos_y": t_y,
      "radius": t_radius,
      "capacity": t_capacity,
      "radius_points": task.radius_points
    }
    print(f"\r#AccessPoints: {len(self.accessPoints)}", end='', flush=True)
    
    for (rx,ry) in task.radius_points: #the radius var of t is already a list of coords
      r = Radius(accessPoint=task)
      self.grid[ry][rx].append(r)
    
    # print(f"Adding one AccessPoint to the grid at {(task.pos_x, task.pos_y)} with radius {t_radius}")
    # print(f"The points within the radius of {t.radius} are \n{t.radius_points}")
    return True
  
  def remove_accessPoint(self, accessPoint:AccessPoint): #, pos_x:int, pos_y:int):
    #remove radius first
    # print(f"radius points to remove are:")
    for r in accessPoint.radius_points:
      # print(f"point:{r[0],r[1]}")
      #get all elements in the cell
      cell_content = self.grid.get_cell_list_contents((r[1],r[0])) 
      # print(f"cell content: {cell_content}")
      for c in cell_content:
      #remove if element is the AccessPoint's radius object
        if isinstance(c, Radius):
          #check if that radius object indeed belong to the same AccessPoint
          if c.accessPoint == accessPoint:
            self.grid[r[1],r[0]].remove(c)
            # print(f"Removing AccessPoint #{accessPoint.unique_id}'s radius point {(r[1],r[0])}")
    #remove AccessPoint
    self.grid[accessPoint.pos_y][accessPoint.pos_x].remove(accessPoint)
    del self.accessPoints[accessPoint]
    # print(f"AccessPoint {accessPoint.unique_id} is removed")

  def step(self):
    """ Advance the model by one step """
    self.schedule.step() #this will call the step() function of each agent
    """We need to trigger the datacollector as well in every step sia"""
    self.datacollector.collect(self)
    """Let's implement here the access point respawn, because it doesn't seem to work 
       properly when called from the agents' side. Probably, removing and adding one at the
       same step does not work always
    """
    access_point_dif = self.num_accessPoints - len(self.accessPoints)
    if(access_point_dif != 0):
      for i in range(access_point_dif):
        self.add_accessPoint()
    

# Visualize

In [ ]:
from mesa.experimental import JupyterViz
import solara
from matplotlib.figure import Figure
import random
from mesa.visualization.UserParam import UserParam


green_to_red_palette=["#FF0000", "#FF7F00","#FFFF00", "#7FFF00", "#00FF00"]

def agent_portrayal(agent):

    if isinstance(agent, SwarmIntelligenceAgent):
        if agent.connected:
            color="tab:pink"
        else:
            color="tab:blue"
        portrayal = {
            "shape": "circle",
            "color": color,
            "filled": "true",
            "layer": 0,
            "r": 1
        }
        

    if isinstance(agent, AccessPoint):
        # print("grid object is AccessPoint")
        # if (agent.radius < 5):
        # #get the capacity of the AccessPoint to know the color
        #     dif_rad = 5 - agent.radius 
        #     color=green_to_red_palette[agent.capacity-1-dif_rad]
        #     # print(f"color id is: {agent.capacity-1+dif_rad}")
        # elif (agent.radius == 5):
        color=green_to_red_palette[agent.capacity-1] # get the color for the actual capacity
        # else:
        #     print("Capacity is bigger than 5, dunno how to color ah")
        #     color="tab:red"
        #     #pass #so far capacity cannot be bigger than 5
        # print(f"color for radius will be: {color}")
        portrayal = {
            "shape": "circle",
            "color": color,
            "filled": "true",
            "layer": 1,
            "r": 3,
            # "w": 0.5,
            # "h": 0.5
        }
    if isinstance(agent, Radius):
        
        portrayal = {
            "shape": "circle",
            "color": agent.accessPoint.radius_color, #color radius nodes according to the random color set in the corresponding accesspoint object
            "filled": "true",
            "layer": 2,
            "r": 1
            # "w":3,
            # "h":3
        }
    # #this color and size if for broken agents
    # size = 30
    # color = "tab:blue"
    # #if an agent is not broken, we use the following color and size
    # if isinstance(agent, AccessPoint):
    # color = "tab:red"
    # size=10
   
    return portrayal


model_params = {
    "num_agents": {
        "type": "SliderInt",
        "value": 6,
        "label": "Number of agents:",
        "min": 1,
        "max": 10,
        "step": 1,
    },
    "grid_x": 30,
    "grid_y": 30,
    # "grid_x": {
    #     "type": "SliderInt",
    #     "value": 15,
    #     "label": "Grid size (x)",
    #     "min": 10,
    #     "max": 200,
    #     "step": 5
    # },
    # "grid_y": {
    #     "type": "SliderInt",
    #     "value": 15,
    #     "label": "Grid size (y)",
    #     "min": 10,
    #     "max": 200,
    #     "step": 5
    # },
    "accessPoint_radius_max": {
        "type": "SliderInt",
        "value": 5,
        "label": "AccessPoint radius",
        "min": 1,
        "max": 5,
        "step": 1
    },
    "num_accessPoints": {
        "type": "SliderInt",
        "value": 4,
        "label": "Number of APs",
        "min": 1,
        "max": 5,
        "step": 1 
    },
    "accessPoint_random_radius": {
        "type":"SliderInt",
        "value": 1,
        "label": "Use random radius for APs",
        "min":0,
        "max":1,
        "step":1
    }
}

page = JupyterViz(
    SwarmIntelligenceModel,
    model_params,
    measures=["Connection Tracker"],
    name="SwarmIntelligence Model",
    agent_portrayal=agent_portrayal,
)
# This is required to render the visualization in the Jupyter notebook
page

#AccessPoints: 4

Cannot show ipywidgets in text

#AccessPoints: 5

# Run multiple times and get some meaningful data

In [67]:
all_num_connected=[]
for j in range(100):
  swarmmodel = SwarmIntelligenceModel(num_agents=10, 
                                    grid_x=50, 
                                    grid_y=50, 
                                    accessPoint_radius_max=5, 
                                    num_accessPoints=5,
                                    accessPoint_random_radius=0)
  num_steps=100
  for i in range(0,num_steps):
    swarmmodel.step()
  
  for agent in swarmmodel.schedule.agents:
    all_num_connected.append(agent.num_times_connected)
    # if(agent.connected):
    #   connected_agents+=1
    # else:
    #   not_connected_agents+=1

print(all_num_connected)
g = sns.histplot(all_num_connected, discrete=True)
g.set(title="Connected status distribution", xlabel="#times conneced", ylabel="Number of agents");

# #prepare data
# data = {
#   "Category": ['Connected Agents', 'Non-connected Agents'],
#   "Count": [connected_agents, not_connected_agents]
# }
# df = pd.DataFrame(data)
# print(df)
# #create barplot
# bar_plot = sns.barplot(x="Category", y="Count", hue="Category", data=df, palette=["green","red"])

# #add title and labels
# # bar_plot.set(style="whitegrid")
# bar_plot.set(title="Connected vs. Non-connected agents", xlabel="Category", ylabel="Count")

# Show the plot
# sns.despine(left=True, bottom=True)
# plt.show()

#AccessPoints: 5[0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 2, 2, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,

# run model with datacolletor module

In [ ]:
model = SwarmIntelligenceModel(num_agents=10, 
                               grid_x=50, 
                               grid_y=50, 
                               accessPoint_radius_max=5, 
                               num_accessPoints=5,
                               accessPoint_random_radius=0)
num_steps=100
for i in range(num_steps):
  model.step()

connected_ratio = model.datacollector.get_model_vars_dataframe()
# print(connected_ratio)

# Set the figure size
plt.figure(figsize=(11, 8))


g=sns.lineplot(data=connected_ratio)
g.set(title=f"Connected ratio over {num_steps} steps", ylabel="Connected agents/All agents");
#set y scale to be between 0-100
plt.ylim(0,100)


# batch run

In [39]:
params = {"num_agents": range(5,11), #evaluate with 1 to 10 agents 
          "grid_x": 50, 
          "grid_y": 50,
          "accessPoint_radius_max": 5,
          "num_accessPoints":5,
          "accessPoint_random_radius":0
          }
results = mesa.batch_run(
  SwarmIntelligenceModel,
  parameters=params,
  iterations=10, #how many times to run each model
  max_steps=100,
  number_processes=1,
  data_collection_period=1,
  display_progress=True
)


  0%|          | 0/60 [00:00<?, ?it/s]

In [64]:
results_df = pd.DataFrame(results)
# results_df.head(50)
#filter out the relevant results only after the 100th step
results_filtered=results_df[(results_df.Step==100)] 
results_filtered[["iteration", "num_agents", "Connection Tracker"]].reset_index(drop=True).head()
g = sns.scatterplot(data=results_filtered, x="num_agents", y="Connection Tracker")
g.set(
    xlabel="Number of agents",
    ylabel="Connected/Not connected",
    title="Connected/Not Connected vs. number of agents",
);